In [19]:
import sympy as syms
import numpy as np
import math

from sympy import symbols
from sympy import Matrix
from sympy import latex
from sympy import solve
from sympy.matrices import eye, zeros, ones, Identity # Special matrices (eye creates explicit matrix, Identity creates matrix expression)
from sympy import *

## Question 1
3-Node Truss Element, with shape function matrix given.
To find:
* Consistent Fully occupied mass matrix
* Lumped Mass Matrix

In [2]:
x,l,row,A = symbols('x L rho A')
N = Matrix([[1 - 3*(x/l) + 2*(x**2/l**2),4*(x/l) - 4*(x**2/l**2),-(x/l) + 2*(x**2/l**2)]])
N # Shape Function Matrix

Matrix([[1 - 3*x/L + 2*x**2/L**2, 4*x/L - 4*x**2/L**2, -x/L + 2*x**2/L**2]])

In [3]:
# Consistent Mass Matrix
M = row*A*syms.integrate(N.T*N,(x,0,l))
M

Matrix([
[2*A*L*rho/15,   A*L*rho/15,  -A*L*rho/30],
[  A*L*rho/15, 8*A*L*rho/15,   A*L*rho/15],
[ -A*L*rho/30,   A*L*rho/15, 2*A*L*rho/15]])

### Since no explicit BCs are given, diagnolisation of the consistent mass matrix is done directly

In [4]:
# Lumped Mass Matrix
MLumped = Matrix([[M[0,0] + M[0,1] + M[0,2],0,0],[0,M[1,0] + M[1,1] + M[1,2],0],[0,0,M[2,0] + M[2,1] + M[2,2]]])
MLumped

Matrix([
[A*L*rho/6,           0,         0],
[        0, 2*A*L*rho/3,         0],
[        0,           0, A*L*rho/6]])

## Question 2

![sys](Images/1.jpg)

**To Find**: The eigenfrequency of the system

**Given Stiffness and Lumped Mass Matrix for a beam element of length $l$**

In [90]:
def stiffness(k):
    E,I,l = symbols('E I l')
    # Length of Element: k*l
    l = k*l
    return (E*I/(l))*Matrix([[12/l**2,6/l,-12/l**2,6/l],[6/l,4,-6/l,2],[-12/l**2,-6/l,12/l**2,-6/l],[6/l,2,-6/l,4]])

def lumpedMass(k):
    rho,A,l = symbols('rho A l')
    return (rho*A*k*l/2)*Matrix([[1,0,0,0],[0,0,0,0],[0,0,1,0],[0,0,0,0]])

In [13]:
# Element 1
K1 = stiffness(2)
M1 = lumpedMass(2)
K1

Matrix([
[ 3*E*I/(2*l**3),  3*E*I/(2*l**2), -3*E*I/(2*l**3),  3*E*I/(2*l**2)],
[ 3*E*I/(2*l**2),         2*E*I/l, -3*E*I/(2*l**2),           E*I/l],
[-3*E*I/(2*l**3), -3*E*I/(2*l**2),  3*E*I/(2*l**3), -3*E*I/(2*l**2)],
[ 3*E*I/(2*l**2),           E*I/l, -3*E*I/(2*l**2),         2*E*I/l]])

In [14]:
# Element 2
K2 = stiffness(1)
M2 = lumpedMass(1)
K2

Matrix([
[ 12*E*I/l**3,  6*E*I/l**2, -12*E*I/l**3,  6*E*I/l**2],
[  6*E*I/l**2,     4*E*I/l,  -6*E*I/l**2,     2*E*I/l],
[-12*E*I/l**3, -6*E*I/l**2,  12*E*I/l**3, -6*E*I/l**2],
[  6*E*I/l**2,     2*E*I/l,  -6*E*I/l**2,     4*E*I/l]])

In [15]:
# Assembled Stiffness Matrix
K = Matrix([[K1[0:2,:],zeros(2,2)],[K1[2:,0:2],K1[2:,2:] + K2[0:2,0:2],K2[0:2,2:]],[zeros(2,2),K2[2:,:]]])
K

Matrix([
[ 3*E*I/(2*l**3),  3*E*I/(2*l**2), -3*E*I/(2*l**3), 3*E*I/(2*l**2),            0,           0],
[ 3*E*I/(2*l**2),         2*E*I/l, -3*E*I/(2*l**2),          E*I/l,            0,           0],
[-3*E*I/(2*l**3), -3*E*I/(2*l**2), 27*E*I/(2*l**3), 9*E*I/(2*l**2), -12*E*I/l**3,  6*E*I/l**2],
[ 3*E*I/(2*l**2),           E*I/l,  9*E*I/(2*l**2),        6*E*I/l,  -6*E*I/l**2,     2*E*I/l],
[              0,               0,    -12*E*I/l**3,    -6*E*I/l**2,  12*E*I/l**3, -6*E*I/l**2],
[              0,               0,      6*E*I/l**2,        2*E*I/l,  -6*E*I/l**2,     4*E*I/l]])

In [16]:
# Assembled Mass Matrix
m = symbols('M')
M = Matrix([[M1[0:2,:],zeros(2,2)],[M1[2:,0:2],M1[2:,2:] + M2[0:2,0:2] + m*Matrix([[1,0],[0,0]]),M2[0:2,2:]],[zeros(2,2),M2[2:,:]]])
M

Matrix([
[A*l*rho, 0,               0, 0,         0, 0],
[      0, 0,               0, 0,         0, 0],
[      0, 0, 3*A*l*rho/2 + M, 0,         0, 0],
[      0, 0,               0, 0,         0, 0],
[      0, 0,               0, 0, A*l*rho/2, 0],
[      0, 0,               0, 0,         0, 0]])

In [17]:
# Reduced System (Node 1 and Node 3 are fixed)
Kred = K[2:4,2:4]
Mred = M[2:4,2:4]
Mred

Matrix([
[3*A*l*rho/2 + M, 0],
[              0, 0]])

In [18]:
# Eigenvalue Problem
lbda = symbols('lambda') # Eigenvalues of the system
evp = Kred - lbda*Mred
evp.det()

(-36*A*E*I*l**4*lambda*rho + 243*E**2*I**2 - 24*E*I*M*l**3*lambda)/(4*l**4)

## Question 3

3 different cases for modelling a structure using 2 beam elements given:
![sys](Images/2.jpg)

### Case 1

In [24]:
# Element 1
K1 = stiffness(Rational(1,2))
M1 = lumpedMass(Rational(1,2))
M1

Matrix([
[A*l*rho/4, 0,         0, 0],
[        0, 0,         0, 0],
[        0, 0, A*l*rho/4, 0],
[        0, 0,         0, 0]])

In [25]:
# Element 2
K2 = stiffness(Rational(1,2))
M2 = lumpedMass(Rational(1,2))
M2

Matrix([
[A*l*rho/4, 0,         0, 0],
[        0, 0,         0, 0],
[        0, 0, A*l*rho/4, 0],
[        0, 0,         0, 0]])

In [23]:
# Assembled Stiffness Matrix
K = Matrix([[K1[0:2,:],zeros(2,2)],[K1[2:,0:2],K1[2:,2:] + K2[0:2,0:2],K2[0:2,2:]],[zeros(2,2),K2[2:,:]]])
K

Matrix([
[ 96*E*I/l**3,  24*E*I/l**2, -96*E*I/l**3,  24*E*I/l**2,            0,            0],
[ 24*E*I/l**2,      8*E*I/l, -24*E*I/l**2,      4*E*I/l,            0,            0],
[-96*E*I/l**3, -24*E*I/l**2, 192*E*I/l**3,            0, -96*E*I/l**3,  24*E*I/l**2],
[ 24*E*I/l**2,      4*E*I/l,            0,     16*E*I/l, -24*E*I/l**2,      4*E*I/l],
[           0,            0, -96*E*I/l**3, -24*E*I/l**2,  96*E*I/l**3, -24*E*I/l**2],
[           0,            0,  24*E*I/l**2,      4*E*I/l, -24*E*I/l**2,      8*E*I/l]])

In [26]:
# Assembled Mass Matrix
m = symbols('M')
M = Matrix([[M1[0:2,:],zeros(2,2)],[M1[2:,0:2],M1[2:,2:] + M2[0:2,0:2],M2[0:2,2:]],[zeros(2,2),M2[2:,:]]])
M

Matrix([
[A*l*rho/4, 0,         0, 0,         0, 0],
[        0, 0,         0, 0,         0, 0],
[        0, 0, A*l*rho/2, 0,         0, 0],
[        0, 0,         0, 0,         0, 0],
[        0, 0,         0, 0, A*l*rho/4, 0],
[        0, 0,         0, 0,         0, 0]])

In [38]:
# Reduced System
# No Trans and Rot DOF for node 1 and no trans DOF for node 3
#Kred = Matrix([[K[2:4,2:4],K[2:4,-1]],[K[-1,2:4],K[-1,-1]]])
A = Matrix([[K[2:4,2:4]],[K[-1,2:4]]])
B = Matrix([[K[2:4,-1]],[K[-1,-1]]])
Kred = Matrix.hstack(A,B)
Kred

Matrix([
[192*E*I/l**3,        0, 24*E*I/l**2],
[           0, 16*E*I/l,     4*E*I/l],
[ 24*E*I/l**2,  4*E*I/l,     8*E*I/l]])

In [39]:
C = Matrix([[M[2:4,2:4]],[M[-1,2:4]]])
D = Matrix([[M[2:4,-1]],[M[-1,-1]]])
Mred = Matrix.hstack(C,D)
Mred

Matrix([
[A*l*rho/2, 0, 0],
[        0, 0, 0],
[        0, 0, 0]])

In [41]:
# Eigenvalue problem
omg = symbols('omega') # Eigenvalues of the system
evp = Kred - (omg**2)*Mred
evp.det()

(-56*A*E**2*I**2*l**4*omega**2*rho + 12288*E**3*I**3)/l**5

### Case 2


In [42]:
# Element 1
K1 = stiffness(Rational(1,3))
M1 = lumpedMass(Rational(1,3))
K1

Matrix([
[ 324*E*I/l**3,  54*E*I/l**2, -324*E*I/l**3,  54*E*I/l**2],
[  54*E*I/l**2,     12*E*I/l,  -54*E*I/l**2,      6*E*I/l],
[-324*E*I/l**3, -54*E*I/l**2,  324*E*I/l**3, -54*E*I/l**2],
[  54*E*I/l**2,      6*E*I/l,  -54*E*I/l**2,     12*E*I/l]])

In [44]:
# Element 2
K2 = stiffness(Rational(2,3))
M2 = lumpedMass(Rational(2,3))
K2

Matrix([
[ 81*E*I/(2*l**3),  27*E*I/(2*l**2), -81*E*I/(2*l**3),  27*E*I/(2*l**2)],
[ 27*E*I/(2*l**2),          6*E*I/l, -27*E*I/(2*l**2),          3*E*I/l],
[-81*E*I/(2*l**3), -27*E*I/(2*l**2),  81*E*I/(2*l**3), -27*E*I/(2*l**2)],
[ 27*E*I/(2*l**2),          3*E*I/l, -27*E*I/(2*l**2),          6*E*I/l]])

In [45]:
# Assembled Stiffness Matrix
K = Matrix([[K1[0:2,:],zeros(2,2)],[K1[2:,0:2],K1[2:,2:] + K2[0:2,0:2],K2[0:2,2:]],[zeros(2,2),K2[2:,:]]])
K

Matrix([
[ 324*E*I/l**3,  54*E*I/l**2,    -324*E*I/l**3,      54*E*I/l**2,                0,                0],
[  54*E*I/l**2,     12*E*I/l,     -54*E*I/l**2,          6*E*I/l,                0,                0],
[-324*E*I/l**3, -54*E*I/l**2, 729*E*I/(2*l**3), -81*E*I/(2*l**2), -81*E*I/(2*l**3),  27*E*I/(2*l**2)],
[  54*E*I/l**2,      6*E*I/l, -81*E*I/(2*l**2),         18*E*I/l, -27*E*I/(2*l**2),          3*E*I/l],
[            0,            0, -81*E*I/(2*l**3), -27*E*I/(2*l**2),  81*E*I/(2*l**3), -27*E*I/(2*l**2)],
[            0,            0,  27*E*I/(2*l**2),          3*E*I/l, -27*E*I/(2*l**2),          6*E*I/l]])

In [46]:
# Assembled Mass Matrix
m = symbols('M')
M = Matrix([[M1[0:2,:],zeros(2,2)],[M1[2:,0:2],M1[2:,2:] + M2[0:2,0:2],M2[0:2,2:]],[zeros(2,2),M2[2:,:]]])
M

Matrix([
[A*l*rho/6, 0,         0, 0,         0, 0],
[        0, 0,         0, 0,         0, 0],
[        0, 0, A*l*rho/2, 0,         0, 0],
[        0, 0,         0, 0,         0, 0],
[        0, 0,         0, 0, A*l*rho/3, 0],
[        0, 0,         0, 0,         0, 0]])

In [47]:
# Reduced System
# No Trans and Rot DOF for node 1 and no trans DOF for node 3
#Kred = Matrix([[K[2:4,2:4],K[2:4,-1]],[K[-1,2:4],K[-1,-1]]])
A = Matrix([[K[2:4,2:4]],[K[-1,2:4]]])
B = Matrix([[K[2:4,-1]],[K[-1,-1]]])
Kred = Matrix.hstack(A,B)
Kred

Matrix([
[729*E*I/(2*l**3), -81*E*I/(2*l**2), 27*E*I/(2*l**2)],
[-81*E*I/(2*l**2),         18*E*I/l,         3*E*I/l],
[ 27*E*I/(2*l**2),          3*E*I/l,         6*E*I/l]])

In [48]:
C = Matrix([[M[2:4,2:4]],[M[-1,2:4]]])
D = Matrix([[M[2:4,-1]],[M[-1,-1]]])
Mred = Matrix.hstack(C,D)
Mred

Matrix([
[A*l*rho/2, 0, 0],
[        0, 0, 0],
[        0, 0, 0]])

In [49]:
# Eigenvalue problem
omg = symbols('omega') # Eigenvalues of the system
evp = Kred - (omg**2)*Mred
evp.det()

(-99*A*E**2*I**2*l**4*omega**2*rho + 39366*E**3*I**3)/(2*l**5)

### Case 3

In [50]:
# Element 1
K1 = stiffness(Rational(2,3))
M1 = lumpedMass(Rational(2,3))
K1

Matrix([
[ 81*E*I/(2*l**3),  27*E*I/(2*l**2), -81*E*I/(2*l**3),  27*E*I/(2*l**2)],
[ 27*E*I/(2*l**2),          6*E*I/l, -27*E*I/(2*l**2),          3*E*I/l],
[-81*E*I/(2*l**3), -27*E*I/(2*l**2),  81*E*I/(2*l**3), -27*E*I/(2*l**2)],
[ 27*E*I/(2*l**2),          3*E*I/l, -27*E*I/(2*l**2),          6*E*I/l]])

In [51]:
# Element 2
K2 = stiffness(Rational(1,3))
M2 = lumpedMass(Rational(1,3))
K2

Matrix([
[ 324*E*I/l**3,  54*E*I/l**2, -324*E*I/l**3,  54*E*I/l**2],
[  54*E*I/l**2,     12*E*I/l,  -54*E*I/l**2,      6*E*I/l],
[-324*E*I/l**3, -54*E*I/l**2,  324*E*I/l**3, -54*E*I/l**2],
[  54*E*I/l**2,      6*E*I/l,  -54*E*I/l**2,     12*E*I/l]])

In [52]:
# Assembled Stiffness Matrix
K = Matrix([[K1[0:2,:],zeros(2,2)],[K1[2:,0:2],K1[2:,2:] + K2[0:2,0:2],K2[0:2,2:]],[zeros(2,2),K2[2:,:]]])
K

Matrix([
[ 81*E*I/(2*l**3),  27*E*I/(2*l**2), -81*E*I/(2*l**3), 27*E*I/(2*l**2),             0,            0],
[ 27*E*I/(2*l**2),          6*E*I/l, -27*E*I/(2*l**2),         3*E*I/l,             0,            0],
[-81*E*I/(2*l**3), -27*E*I/(2*l**2), 729*E*I/(2*l**3), 81*E*I/(2*l**2), -324*E*I/l**3,  54*E*I/l**2],
[ 27*E*I/(2*l**2),          3*E*I/l,  81*E*I/(2*l**2),        18*E*I/l,  -54*E*I/l**2,      6*E*I/l],
[               0,                0,    -324*E*I/l**3,    -54*E*I/l**2,  324*E*I/l**3, -54*E*I/l**2],
[               0,                0,      54*E*I/l**2,         6*E*I/l,  -54*E*I/l**2,     12*E*I/l]])

In [53]:
# Assembled Mass Matrix
m = symbols('M')
M = Matrix([[M1[0:2,:],zeros(2,2)],[M1[2:,0:2],M1[2:,2:] + M2[0:2,0:2],M2[0:2,2:]],[zeros(2,2),M2[2:,:]]])
M

Matrix([
[A*l*rho/3, 0,         0, 0,         0, 0],
[        0, 0,         0, 0,         0, 0],
[        0, 0, A*l*rho/2, 0,         0, 0],
[        0, 0,         0, 0,         0, 0],
[        0, 0,         0, 0, A*l*rho/6, 0],
[        0, 0,         0, 0,         0, 0]])

In [54]:
# Reduced System
# No Trans and Rot DOF for node 1 and no trans DOF for node 3
#Kred = Matrix([[K[2:4,2:4],K[2:4,-1]],[K[-1,2:4],K[-1,-1]]])
A = Matrix([[K[2:4,2:4]],[K[-1,2:4]]])
B = Matrix([[K[2:4,-1]],[K[-1,-1]]])
Kred = Matrix.hstack(A,B)
Kred

Matrix([
[729*E*I/(2*l**3), 81*E*I/(2*l**2), 54*E*I/l**2],
[ 81*E*I/(2*l**2),        18*E*I/l,     6*E*I/l],
[     54*E*I/l**2,         6*E*I/l,    12*E*I/l]])

In [55]:
C = Matrix([[M[2:4,2:4]],[M[-1,2:4]]])
D = Matrix([[M[2:4,-1]],[M[-1,-1]]])
Mred = Matrix.hstack(C,D)
Mred

Matrix([
[A*l*rho/2, 0, 0],
[        0, 0, 0],
[        0, 0, 0]])

In [56]:
# Eigenvalue problem
omg = symbols('omega') # Eigenvalues of the system
evp = Kred - (omg**2)*Mred
evp.det()

(-90*A*E**2*I**2*l**4*omega**2*rho + 19683*E**3*I**3)/l**5

## Question 4

3 Node truss element with node 1 fixed.
2nd Order element so the shape functions will be quadratic.
Shape functions belong to the *Lagrange Family*

In [83]:
x,a,E = symbols('x A E')
# Shape Function Vector
A = 2*(x**2/l**2) - 3*(x/l) + 1
B = 4*(x/l) - 4*(x**2/l**2)
C = 2*(x**2/l**2) - x/l
N = Matrix([[A,B,C]])
N

Matrix([[1 - 3*x/L + 2*x**2/L**2, 4*x/L - 4*x**2/L**2, -x/L + 2*x**2/L**2]])

In [78]:
# Consistent Mass Matrix
M = row*a*syms.integrate(N.T*N,(x,0,l))
M

int

In [79]:
# Function for lumping of a consistent mass matrix
def lumping(Mcon):
    Mlumped = zeros(Mcon.shape[0])
    for i in range(0,Mcon.shape[0]):
        Mlumped[i,i] = np.sum(Mcon[i,:])

    return Mlumped

In [80]:
# Lumped Mass Matrix
Mlump = lumping(M)
Mlump

Matrix([
[A*L*rho/6,           0,         0],
[        0, 2*A*L*rho/3,         0],
[        0,           0, A*L*rho/6]])

In [85]:
# Given Stiffness Matrix
K = (E*a/(3*l))*Matrix([[7,-8,1],[-8,16,-8],[1,-8,7]])
K

Matrix([
[ 7*A*E/(3*L), -8*A*E/(3*L),    A*E/(3*L)],
[-8*A*E/(3*L), 16*A*E/(3*L), -8*A*E/(3*L)],
[   A*E/(3*L), -8*A*E/(3*L),  7*A*E/(3*L)]])

In [88]:
# Reduced System (Node 1 is fixed)
Kred = K[1:,1:]
Mred = Mlump[1:,1:]
Mred

Matrix([
[2*A*L*rho/3,         0],
[          0, A*L*rho/6]])

In [89]:
# Eigenvalue Problem
omg = symbols('omega')
evp = Kred - (omg**2)*Mred
evp.det()

(48*A**2*E**2 - 22*A**2*E*L**2*omega**2*rho + A**2*L**4*omega**4*rho**2)/(9*L**2)

## Question 5

![sys](Images/3.jpg)

In [97]:
# Reduced Stiffness Matrix
K1 = stiffness(1)
K2 = stiffness(1)
K = Matrix([[K1[0:2,:],zeros(2,2)],[K1[2:,0:2],K1[2:,2:] + K2[0:2,0:2],K2[0:2,2:]],[zeros(2,2),K2[2:,:]]]) # Assembled Stiffness
Kred = K[2:,2:] # Node 1 is fixed
Kred

Matrix([
[ 24*E*I/l**3,           0, -12*E*I/l**3,  6*E*I/l**2],
[           0,     8*E*I/l,  -6*E*I/l**2,     2*E*I/l],
[-12*E*I/l**3, -6*E*I/l**2,  12*E*I/l**3, -6*E*I/l**2],
[  6*E*I/l**2,     2*E*I/l,  -6*E*I/l**2,     4*E*I/l]])

In [99]:
# Reduced Lumped Mass Matrix
M1 = lumpedMass(1)
M2 = lumpedMass(1)
M = Matrix([[M1[0:2,:],zeros(2,2)],[M1[2:,0:2],M1[2:,2:] + M2[0:2,0:2],M2[0:2,2:]],[zeros(2,2),M2[2:,:]]]) # Assembled Mass
Mred = M[2:,2:]
Mred

Matrix([
[A*l*rho, 0,         0, 0],
[      0, 0,         0, 0],
[      0, 0, A*l*rho/2, 0],
[      0, 0,         0, 0]])

In [100]:
# Eigenvalue problem
evp = Kred - lbda*Mred
evp.det()

(14*A**2*E**2*I**2*l**8*lambda**2*rho**2 - 240*A*E**3*I**3*l**4*lambda*rho + 144*E**4*I**4)/l**8

## Question 6

![sys](Images/4.jpg)

Eigenfrequency of the structure to be computed

In [123]:
# Beam Element
mass,m,rho,A,l = symbols('M m rho A l')
Kbeam = stiffness(1)
Mbeam = lumpedMass(1)
Mbeam = (m/(rho*A*l))*lumpedMass(1)
Kbeam

Matrix([
[ 12*E*I/l**3,  6*E*I/l**2, -12*E*I/l**3,  6*E*I/l**2],
[  6*E*I/l**2,     4*E*I/l,  -6*E*I/l**2,     2*E*I/l],
[-12*E*I/l**3, -6*E*I/l**2,  12*E*I/l**3, -6*E*I/l**2],
[  6*E*I/l**2,     2*E*I/l,  -6*E*I/l**2,     4*E*I/l]])

In [120]:
# Spring Element
k = symbols('k')
Kspring = k*Matrix([[1,-1],[-1,1]])
Mspring = (m/2)*Matrix([[1,0],[0,1]])
Kspring

Matrix([
[ k, -k],
[-k,  k]])

In [121]:
# Assembled System 
K = Matrix.hstack(Kbeam,zeros(Kbeam.shape[0],1))
K = Matrix.vstack(K,zeros(1,K.shape[1]))
K[2,2] = K[2,2] + Kspring[0,0]
K[2,-1] = Kspring[0,-1]
K[-1,2] = Kspring[-1,0]
K[-1,-1] = Kspring[-1,-1]
K

Matrix([
[ 12*E*I/l**3,  6*E*I/l**2,    -12*E*I/l**3,  6*E*I/l**2,  0],
[  6*E*I/l**2,     4*E*I/l,     -6*E*I/l**2,     2*E*I/l,  0],
[-12*E*I/l**3, -6*E*I/l**2, 12*E*I/l**3 + k, -6*E*I/l**2, -k],
[  6*E*I/l**2,     2*E*I/l,     -6*E*I/l**2,     4*E*I/l,  0],
[           0,           0,              -k,           0,  k]])

In [124]:
M = Matrix.hstack(Mbeam,zeros(Mbeam.shape[0],1))
M = Matrix.vstack(M,zeros(1,M.shape[1]))
M[2,2] = M[2,2] + Mspring[0,0] + mass
M[2,-1] = Mspring[0,-1]
M[-1,2] = Mspring[-1,0]
M[-1,-1] = Mspring[-1,-1]
M

Matrix([
[m/2, 0,     0, 0,   0],
[  0, 0,     0, 0,   0],
[  0, 0, M + m, 0,   0],
[  0, 0,     0, 0,   0],
[  0, 0,     0, 0, m/2]])

In [127]:
# Reduced System
# Node 1 and Node 3 are both fixed
Kred = K[2:4,2:4]
Mred = M[2:4,2:4]
Mred

Matrix([
[M + m, 0],
[    0, 0]])

In [128]:
# Eigenvalue Problem
evp = Kred - (omg**2)*Mred
evp.det()

(12*E**2*I**2 - 4*E*I*M*l**3*omega**2 + 4*E*I*k*l**3 - 4*E*I*l**3*m*omega**2)/l**4